In [25]:
import yaml
def load_config(config_file):
    with open(config_file, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

In [26]:
from collections import namedtuple
config = load_config('settings_detector_xwav_default.yaml')
# detParams = namedtuple('detParams', config.keys())(*config.values())
# detParams = DetParams(**config)
detParams = type('DetParams',(object,),config)()

In [27]:
from wavio import *
from functions import *
from detection import *

In [28]:
dt_buildDirs(detParams)

In [29]:
fullFileNames = fn_findXWAVs(detParams.baseDir, 'x.wav')
fullLabels = fn_getFileset(detParams.metaDir, fullFileNames)
fullFileNames, fullLabels

(['/Users/Jim/Documents/Project/WhaleClassification/WAT_NC_01/Arctic_C2_10_150728_020000.x.wav'],
 ['/Users/Jim/Documents/Project/WhaleClassification/output/Arctic_C2_10_150728_020000_metadata/Arctic_C2_10_150728_020000.c'])

In [30]:
currentRecFile = fullFileNames[0]
hdr = io_readXWavHeader(currentRecFile)
vars(hdr)

{'end': <wavio.start at 0x10f324e90>,
 'fType': 'xwav',
 'fs': 200000,
 'nBits': 16,
 'nch': 1,
 'raw': <wavio.raw at 0x10f324810>,
 'samp': <wavio.sample at 0x1070c6710>,
 'start': <wavio.start at 0x10f324f90>,
 'xgain': 1,
 'xhd': <wavio.xhd at 0x10f18cbd0>}

In [31]:
startsSec, stopsSec = dt_chooseSegmentsRaw(hdr)
startsSec, stopsSec

(array([   0.        ,   75.00000001,  150.00000002,  225.00000003,
         300.00000003,  375.00000004,  450.00000005,  525.00000006,
         900.00000003,  975.00000003, 1050.00000004, 1125.00000005,
        1200.00000006, 1274.99999999, 1350.        , 1425.00000001,
        1800.00000005, 1875.00000006, 1949.99999999, 2025.        ,
        2100.00000001, 2175.00000002, 2250.00000003, 2325.00000003,
        2700.        , 2775.00000001, 2850.00000002, 2925.00000003,
        3000.00000003, 3075.00000004]),
 array([  74.99999506,  149.99999499,  224.999995  ,  299.99999501,
         374.99999501,  449.99999502,  524.99999503,  599.99999504,
         974.99999501, 1049.99999501, 1124.99999502, 1199.99999503,
        1274.99999504, 1349.99999505, 1424.99999506, 1499.99999499,
        1874.99999503, 1949.99999504, 2024.99999505, 2099.99999506,
        2174.99999499, 2249.999995  , 2324.99999501, 2399.99999501,
        2774.99999506, 2849.99999499, 2924.999995  , 2999.99999501,
        

In [32]:
previousFs, _ = fn_buildFilters(detParams, hdr.fs)
# detParams.previousFs = previousFs
params = fn_interp_tf(detParams)
if 'countThresh' not in vars(detParams) or detParams.countThresh == 0:
    detParams.countThresh = (10**((detParams.dBppThreshold - np.median(detParams.xfrOffset))/20))/2

--ends in *dt_batch* #57 ---

## Starts from here

In [33]:
from __future__ import division
import numpy as np
from scipy.signal import filtfilt

cParams = dt_init_cParams(detParams) # set up storage for HR output.
sIdx = 1
# Open audio file
with open(currentRecFile, 'r') as fh:
    buffSamples = int(detParams.LRbuffer*hdr.fs)
    # Loop through search area, running short term detectors
    detParams.clickSampleLims = np.int64(np.ceil((hdr.fs/10**6)*\
                                         np.array([detParams.delphClickDurLims[0]*.75,\
                                         detParams.delphClickDurLims[1]*1.25])))
    
    # [TODO] k for loop
    k =0
    startK = startsSec[k]
    stopK = stopsSec[k]
    
    # [TODO] Read in data segment for WAV format
    dat = io_readRaw(fh, hdr, k, detParams.channel)
    
    # bandpass
    if detParams.filterSignal:
        padlen = 3*(max(len(detParams.fB),len(detParams.fA))-1)
        filtData = filtfilt(detParams.fB, detParams.fA, dat, padtype='odd', padlen=padlen)
    else:
        filtData = dat
        
    energy = filtData ** 2
    
    ### Run LR detection to identify candidates
    detectionsSample,detectionsSec =  dt_LR(energy,hdr,buffSamples,startK,stopK,detParams)
    
    # [2/9]
    # [FUTURE] PARALLEL in detectionsSample
    for iD in range(len(detectionsSample)):
        # [FUTURE] HOW TO EXTRACT SUBARRAY FROM NP ARRAY WITH DIFFERENT LENGTH
        filtSegment = filtData[detectionsSample[iD][0]:detectionsSample[iD][1]+1]
        clicks, noise = dt_HR(detParams, hdr, filtSegment)
        print clicks, noise

15000000
[[0. 0.]] [170.0, 470.0]
[[475. 574.]] [150.0, 450.0]
[[424. 690.]] [150.0, 450.0]
[[470. 573.]] [150.0, 450.0]
[[423. 539.]] [150.0, 450.0]
[[0. 0.]] [150.0, 450.0]
[[479. 636.]] [150.0, 450.0]
[[420. 596.]] [150.0, 450.0]
[[488. 572.]] [150.0, 450.0]
[[466. 664.]] [150.0, 450.0]
[[481. 629.]] [150.0, 450.0]
[[476. 633.]] [150.0, 450.0]
[[468. 652.]] [150.0, 450.0]
[[472. 617.]] [150.0, 450.0]
[[472. 571.]] [150.0, 450.0]
[[474. 581.]] [150.0, 450.0]
[[ 465.  657.]
 [1460. 1735.]] [705.0, 1005.0]
[[467. 661.]] [150.0, 450.0]
[[460. 588.]] [150.0, 450.0]
[[463. 594.]] [150.0, 450.0]
[[475. 736.]] [150.0, 450.0]
[[ 466.  573.]
 [1062. 1253.]] [650.0, 950.0]
[[463. 606.]] [150.0, 450.0]
[[470. 524.]
 [732. 849.]] [150.0, 450.0]
[[476. 608.]] [150.0, 450.0]
[[464. 612.]] [150.0, 450.0]
[[ 474.  622.]
 [1326. 1444.]
 [   0.    0.]
 [4276. 4421.]
 [5334. 5493.]] [3040.0, 3340.0]
[[455. 567.]] [150.0, 450.0]
[[474. 540.]] [150.0, 450.0]
[[446. 523.]] [150.0, 450.0]
[[490. 778.]] [15

start HR detection on candidates